# Learn to differentiate and anti-differentiate

## About
The stencil includes four types of questions, demonstrated below.

1. Derivatives of power functions 
2. Anti-derivatives of power functions
3. Derivatives of exponential functions 
4. Anti-derivatives of exponential functions

The four question types... | with answers...
-|-
![](questions.png) | ![](answers.png)

## Usage
You can change details below, when you are ready simply use the `Run` menu and select `Run All Cells`, then navigate to the output folder using the Files tab on the left, and open the generated stencil PDFs. You can right click and download them for printing.

In [1]:
# Personalize stencils...
classes = [
    {
        'name': 'Class 1',
        'students': [
            'Ada Lovelace',
            'Grace Hopper',
            'Jean Jennings Bartik',
        ],
    }
]

# One sheet (front and back page) for the questions, and one sheet for the answers will fit using 75 exercises
exercises = 75

filename = "deriv-antideriv"
topic_questions = "Derivatives and anti-derivatives"
topic_solutions = topic_questions + " (Solutions)"
output_directory = "output"

# Stencil generation code

In [2]:
# Useful tools for generating exercises
import random
from sympy import *
init_printing()

In [3]:
# These are the symbols that I will use
var = symbols('x t')
int_const = symbols('C')

# Some sets of constants to be used at random to create exercises
# Symbols are assumed to be non-zero to avoid "if ... then ..." in the solutions
# Symbols are assumed to be integers to avoid partial derivatives symbols being generated by Sympy
num = [S(1), S(2), S(3), S(4), S(5), symbols('a', nonzero=True, integer=True)]
denom = [S(1), S(2), S(3), S(4), S(5), symbols('b', nonzero=True, integer=True)]

exp_base = [S(2), S(3), S(4), S(5), pi, E]
exp_coef_sign = [S(1), S(1), S(1), S(-1)]
exp_coef_num = [S(1), S(2), S(3), S(4), S(5), symbols('k', nonzero=True, integer=True)]
exp_coef_denom = [S(1), S(1), S(1), S(1), S(1), S(1), S(2), S(3)]

pow_exp_sign = [S(1), S(1), S(1), S(-1)]
pow_exp_num = [S(0), S(1), S(2), S(3), S(4), symbols('n', nonzero=True, integer=True)]
pow_exp_denom = [S(1), S(1), S(1), S(1), S(1), S(1), S(2), S(3)]

def power():
    variable = random.choice(var)

    expression = simplify(random.choice(num) / random.choice(denom))
    expression *= variable ** simplify(random.choice(pow_exp_sign) * random.choice(pow_exp_num) / random.choice(pow_exp_denom))
    return expression, variable

def exponential():
    variable = random.choice(var)

    expression = simplify(random.choice(num) / random.choice(denom))
    expression *= random.choice(exp_base) ** simplify(random.choice(exp_coef_sign) * random.choice(exp_coef_num) / random.choice(exp_coef_denom) * variable)
    return expression, variable

def gen_antideriv(func):
    expression, variable = func()   
    q = Integral(expression, variable)
    a = q.doit() + int_const
    return q, a

def gen_deriv(func):
    expression, variable = func()    
    q = Derivative(expression, variable)
    a = q.doit()
    return q, a

In [4]:
# Select exercises...
generators = [
    lambda: gen_deriv(power),
    lambda: gen_antideriv(power),
    lambda: gen_deriv(exponential),
    lambda: gen_antideriv(exponential)
]

In [5]:
pre_latex = r"""
\documentclass[12pt, a4paper, fleqn]{article}

\usepackage[margin=1in]{geometry} 
\usepackage{amsmath,amsthm,amssymb,amsfonts}

% resets the equation numbering in between sections
\usepackage{chngcntr}
\counterwithin*{equation}{section}

\newcommand{\stencil}[4]{
    \twocolumn[
        \begin{@twocolumnfalse}
            \heading{#1}{#2}{#3}
        \end{@twocolumnfalse}
    ]
    #4
}

\newcommand{\heading}[3]{
    \section{#1 - #2 - #3}
}

\newcommand{\question}[1]{
    \begin{equation}
        #1 \, =
    \end{equation}
}

\newcommand{\answer}[1]{
    \begin{equation}
        #1
    \end{equation}
}

\begin{document}
"""

post_latex = r"""
\end{document}
"""

In [6]:
def question(q):
    return r"\question{{{q:}}}".format(q=latex(q)) + '\n'

def answer(a):
    return r"\answer{{{a:}}}".format(a=latex(a)) + '\n'

def stencil(**kvargs):
    return r"\stencil{{{cls:}}}{{{name:}}}{{{topic:}}}{{\n{body:}}}".format(**kvargs) + '\n'

In [7]:
def progress(cls, cur, tot):
    print('Generating exercises for ' + cls + ': [' + '#'*cur + ' '*(tot-cur) + ']', end='\r', flush=True)

In [8]:
# Run this to generate a set of stencils for each student in each class
# The PDF's are saved in the output folder

for cls in classes:
    students = []
    qfile = "{}-{}-q".format(filename, cls['name'])
    afile = "{}-{}-a".format(filename, cls['name'])
    
    student_count = len(cls['students'])

    for i, student_name in enumerate(cls['students']):
        progress(cls['name'], i, student_count)
        
        q_body = ''
        a_body = ''
        for i in range(exercises):
            q, a = random.choice(generators)()
            q_body += question(q)
            a_body += answer(a)
            
            
        student = {
            'q_body': stencil(topic=topic_questions, body=q_body, name=student_name, cls=cls['name']),
            'a_body': stencil(topic=topic_solutions, body=a_body, name=student_name, cls=cls['name']),
        }
        students.append(student)

    with open(qfile + ".tex", 'w') as file:
        file.write(pre_latex)

        for student in students:
            file.write(student['q_body'])

        file.write(post_latex)

    with open(afile + ".tex", 'w') as file:
        file.write(pre_latex)

        for student in students:
            file.write(student['a_body'])

        file.write(post_latex)    

    !mkdir -p output
    print('Writing ' + qfile + '.pdf ...')
    !pdflatex -interaction=batchmode "{qfile}.tex" > /dev/null
    print('Writing ' + afile + '.pdf ...')
    !pdflatex -interaction=batchmode "{afile}.tex" > /dev/null
    !mv "{qfile}.pdf" "{afile}.pdf" "{output_directory}/"
    !rm "{qfile}."* "{afile}."* "texput.log" 2> /dev/null
    
print('Done!')

Writing deriv-antideriv-Class 1-q.pdf ...
Writing deriv-antideriv-Class 1-a.pdf ...
Done!
